In [2]:
import os
import torch

from datasets import load_dataset
from transformers import (
    
    BitsAndBytesConfig,
    HfArgumentParser,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq,AutoModelForSeq2SeqLM

d:\Python Projects\Llama 2 fine tuning\venv\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [16]:


model_name = "distilgpt2"

# Custom data for fine-tuning
custom_data = [
    {"instruction": "Tell me about your experience in data science.", "output": "I have over 5 years of experience in data science, working on various projects involving machine learning, deep learning, and data analysis."},
    {"instruction": "What are your hobbies?", "output": "I enjoy hiking, reading, and playing the guitar in my free time."},
    {"instruction": "What is your educational background?", "output": "I have a Master's degree in Computer Science with a specialization in Artificial Intelligence."},
    {"instruction": "Describe your favorite project.", "output": "My favorite project involved developing a recommendation system for an e-commerce platform, which significantly improved user engagement and sales."}
]

# Convert custom data to dataset
dataset = Dataset.from_list(custom_data)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model =  AutoModelForCausalLM.from_pretrained(model_name)

# Set pad_token if it is not set
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))

# Test the model with an input text before fine-tuning
input_text = "Hello, Tell me about youself?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate output using the pre-trained model
pre_finetuning_output = model.generate(input_ids)
pre_finetuning_output_text = tokenizer.decode(pre_finetuning_output[0], skip_special_tokens=True)

print("Response before fine-tuning:", pre_finetuning_output_text)

# Tokenize function
def tokenize_function(examples):
    inputs = examples['instruction']
    targets = examples['output']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length').input_ids
    model_inputs["labels"] = labels
    return model_inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Data
# Data collator for padding and creating batches
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=10,
    save_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_total_limit=1,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Test the model with the same input text after fine-tuning
post_finetuning_output = model.generate(input_ids, max_length=50)  # Adjust max_length as needed
post_finetuning_output_text = tokenizer.decode(post_finetuning_output[0], skip_special_tokens=True)

print("Response after fine-tuning:", post_finetuning_output_text)




Using pad_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response before fine-tuning: Hello, Tell me about youself?

I’m a very young woman, and


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'train_runtime': 24.3611, 'train_samples_per_second': 0.164, 'train_steps_per_second': 0.041, 'train_loss': 10.345245361328125, 'epoch': 1.0}
Response after fine-tuning: Hello, Tell me about youself?

I'm a big fan of the game, and I'm a big fan of the game, and I'm a big fan of the game, and I'm a big fan of the game, and


In [14]:
output_dir="./results"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./results\\tokenizer_config.json',
 './results\\special_tokens_map.json',
 './results\\vocab.json',
 './results\\merges.txt',
 './results\\added_tokens.json',
 './results\\tokenizer.json')